In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [13]:
params = list(net.parameters())
print(len(params))
print(params[0].size())   # conv1's weight size -- 6 filters per channel(i.e. outputs), 3x3 kernel_size
print(params[1].size())   # activation layer should print output of conv layer, which is 6
print(params[-1].size())  # output layer should print 10 outputs

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([10])


## Forward Pass

In [23]:
device = torch.device('cuda')
input_ = torch.randn(1, 1, 32, 32, requires_grad=True)
input_.to(device)
out = net(input_)
out

tensor([[-0.0612, -0.0631, -0.0696, -0.0492,  0.0608,  0.1006, -0.1260, -0.0427,
          0.0454, -0.0384]], grad_fn=<AddmmBackward>)

## Backpropagate

In [31]:
net.zero_grad()
out.backward(torch.randn(1, 10))   # backprop using a random input

### Backpropagate with loss function

In [35]:
net.zero_grad()
criterion = nn.MSELoss()

output = net(input_)
target = torch.randn(10)
target = target.view(1, -1)
loss = criterion(output, target)
loss

tensor(1.4516, grad_fn=<MseLossBackward>)

`loss.grad_fn` stores a directed graph of gradient functions for step in the forward pass. (i.e. MSELoss -> Linear (Adam) --> ReLU (AccumulateGrad)

In [39]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Storing the updated gradients to the parameters

In [40]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0151, -0.0231, -0.0334, -0.0146, -0.0194, -0.0089])


## Applying the updated gradients

In [41]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input_)
loss = criterion(output, target)
loss.backward()
optimizer.step()   # updates the network

## Put all tensor parameters on CUDA

In [42]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# model's parameters on CUDA
net.to(device)

cuda:0


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [43]:
## dataset's inputs and target on CUDA
input_, target = input_.to(device), target.to(device)